In [4]:
from requests.auth import HTTPBasicAuth
from datetime import datetime, timezone
import shapely
from shapely import Point, Polygon, MultiPolygon
import yaml
import ipaddress
import os

import geopandas as gpd
import pandas as pd
from sentinelsat.sentinel import SentinelAPI

from aimlsse_api.client import SatelliteDataClient, GroundDataClient
from aimlsse_api.data import Credentials

Make sure that the file `login.yml` exists. If not, create a new one and store your credentials for the [Copernicus OpenAPI Hub](https://scihub.copernicus.eu/dhus/#/home) in the following form:
```
username: <copernicus-username>
password: <copernicus-password>
```
The file is excluded from git to make sure that credentials are kept private.

In [5]:
login_config = yaml.safe_load(open('login.yml'))
username = login_config['username']
password = login_config['password']
copernicus_login = Credentials(username, password)

In [8]:
data_dir = 'data'

In [6]:
area_of_interest = gpd.GeoDataFrame.from_file('input/USA.geojson')
print(area_of_interest.crs)
polygon = area_of_interest.iloc[0]['geometry']

epsg:4326


In [15]:
satellite_client = SatelliteDataClient(ipaddress.ip_address('127.0.0.1'), 8010)
products_metadata_filename = 'products_metadata.csv'
products_metadata_filepath = os.path.join(data_dir, products_metadata_filename)
if os.path.exists(products_metadata_filepath):
    products = pd.read_csv(products_metadata_filepath, index_col=['id'])
else:
    products = satellite_client.queryProductsMetadata(
        polygon,
        datetime(2023, 1, 1, tzinfo=timezone.utc),
        datetime(2023, 2, 1, tzinfo=timezone.utc),
        copernicus_login
    )
    products.to_csv(products_metadata_filepath, index_label='id')
print(products.columns)

Index(['title', 'link', 'link_alternative', 'link_icon', 'summary', 'ondemand',
       'datatakesensingstart', 'generationdate', 'beginposition',
       'endposition', 'ingestiondate', 'orbitnumber', 'relativeorbitnumber',
       'cloudcoverpercentage', 'sensoroperationalmode', 'gmlfootprint',
       'footprint', 'level1cpdiidentifier', 'tileid', 'hv_order_tileid',
       'format', 'processingbaseline', 'platformname', 'filename',
       'instrumentname', 'instrumentshortname', 'size', 's2datatakeid',
       'producttype', 'platformidentifier', 'orbitdirection',
       'platformserialidentifier', 'processinglevel', 'datastripidentifier',
       'granuleidentifier', 'identifier', 'uuid'],
      dtype='object')


In [16]:
products['generationdate'] = pd.to_datetime(products['generationdate'])
products_sorted = products.sort_values(by=['cloudcoverpercentage', 'generationdate'], ascending=[False, True])
print(products_sorted[['cloudcoverpercentage', 'generationdate']])

                                      cloudcoverpercentage  \
id                                                           
aa5d9a34-d4e6-4c5e-8a56-f1eb696741a0                 100.0   
2b935353-afde-4fd7-b42d-d7146ac003af                 100.0   
8a601697-f469-407c-94d6-ad05da5b8e29                 100.0   
1ae3a87c-4648-4764-92f1-3a4d4982dcb4                 100.0   
e2b2d43d-7430-4de5-bf53-c72c444f9cc5                 100.0   
...                                                    ...   
86c7f37d-8924-43c3-b65c-2cd3d7b13417                   0.0   
82960a02-f7d0-4374-980a-e79084883638                   0.0   
c6d1d326-0276-4201-b6c1-ec4e056ae469                   0.0   
b07d8a6c-9ffc-46fd-bc59-6bdd1459d7a1                   0.0   
16b4bbe0-23b5-4e0e-82ee-e2ff32fe44d6                   0.0   

                                                 generationdate  
id                                                               
aa5d9a34-d4e6-4c5e-8a56-f1eb696741a0 1970-01-01 00:27:52.5966

In [17]:
products_range_selection = products_sorted[(products_sorted['cloudcoverpercentage'] > 50) & (products_sorted['cloudcoverpercentage'] < 70)]

In [18]:
print(products_range_selection['cloudcoverpercentage'])

id
405c25dc-37a3-45bd-b07c-1dce4c56b138    69.988743
dd286f3f-16c7-4c0e-98cd-4051ed536db5    69.985661
2e0355d0-109f-4282-80ed-9eb459e5d180    69.932187
939b9586-a20d-4a2f-8aac-8e3d55b6b7e4    69.886648
742aeed4-20d2-4132-9612-8ce57aa9e163    69.828357
                                          ...    
28d6373b-0682-435c-a720-4cfe6229b04e    50.040999
927d999b-984f-4980-909e-ac6c0b9a069c    50.040060
6e6228ef-aad4-4836-a13e-5fb2e95dcf1d    50.036848
e198f895-0077-483b-886a-8691caff3f67    50.016704
1c37a07e-34df-4462-9e3c-831a72667f89    50.006792
Name: cloudcoverpercentage, Length: 1115, dtype: float64


In [19]:
print(products_range_selection.loc['405c25dc-37a3-45bd-b07c-1dce4c56b138']['footprint'])
multi_poly: MultiPolygon = shapely.from_wkt(products_range_selection.loc['405c25dc-37a3-45bd-b07c-1dce4c56b138']['footprint'])

MULTIPOLYGON (((-101.67325 33.306789832353964, -101.634735 34.29540573036853, -102.82666 34.32237324440533, -102.85156 33.332774540686216, -101.67325 33.306789832353964)))


In [20]:
ground_client = GroundDataClient(ipaddress.ip_address('127.0.0.1'), 8000)
stations = ground_client.queryMetadata(
    polygons=[multi_poly]
)
print(stations)

                        geometry    id                     name  \
0    POINT (-101.81400 33.65400)   LBB                  Lubbock   
1    POINT (-102.00000 33.60000)  KREE  Reese Afb/Lubock  TX/US   
2    POINT (-102.37250 33.55250)   LLN                Levelland   
3    POINT (-101.82278 33.66364)   LBB        LUBBOCK INTL ARPT   
4    POINT (-101.71734 34.16815)   PVW                PLAINVIEW   
..                           ...   ...                      ...   
102  POINT (-102.12000 34.09000)  XONS                    OLTON   
103  POINT (-101.71000 34.18000)  XPVS                PLAINVIEW   
104  POINT (-102.05000 33.61000)  XREE                    REESE   
105  POINT (-102.61000 33.39000)  XSDS                  SUNDOWN   
106  POINT (-102.05000 33.42000)  XWOS                WOLFFORTH   

                   plot_name  network  latitude  longitude  elevation  \
0                       None   NEXRAD  33.65400 -101.81400  1029.0000   
1    Reese Afb/Lubock, TX/US     RAOB  33.60000 -

In [21]:
print(stations[['id', 'geometry']])

       id                     geometry
0     LBB  POINT (-101.81400 33.65400)
1    KREE  POINT (-102.00000 33.60000)
2     LLN  POINT (-102.37250 33.55250)
3     LBB  POINT (-101.82278 33.66364)
4     PVW  POINT (-101.71734 34.16815)
..    ...                          ...
102  XONS  POINT (-102.12000 34.09000)
103  XPVS  POINT (-101.71000 34.18000)
104  XREE  POINT (-102.05000 33.61000)
105  XSDS  POINT (-102.61000 33.39000)
106  XWOS  POINT (-102.05000 33.42000)

[107 rows x 2 columns]


In [22]:
stations.to_file('data/product_stations.geojson')